<a href="https://colab.research.google.com/github/VIS-WA/Duplicate-Question-Detection-in-Stack-Overflow/blob/main/Copy_of_Primary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/study"

/content/drive/MyDrive/study


## Libraries

In [3]:
# libraries
import nltk #NLP lib
import numpy as np # numpy for arrays
import pandas as pd # pandas lib for data handling
from nltk.corpus import stopwords # common eng sstopwords
from nltk.stem import PorterStemmer #porter stemmer algo
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup # HTML to txt
import os
import sys
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
ps = PorterStemmer()
sw_nltk = stopwords.words('english') # bag of all common english stop words
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### contractions

In [ ]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

## Supporting Functions

In [4]:
import re
def TokStem(text,j): # Tokenise and Stem the given text
    stoplist = set(nltk.corpus.stopwords.words('english'))
    # text = [contraction[word.islower()] for word in text.split(" ") if word.islower() in contractions.keys() ]
    # print(text)
    
    text =  nltk.word_tokenize(text.lower())
    # text = re.split(',text)    
    # print(text)
    
    words = text #[word for word in text if word.isalpha()]
    ltz = WordNetLemmatizer()
    # words = [word for word in text.split() if word.lower() not in sw_nltk] # bag of tokenised words
    final1 = []
    final = []
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~``'''
    contract_strings = ["'s", "\"","'ve"]
    for word in words:
      if word not in stoplist and word not in punc and word not in contract_strings and word!='"':
        
        final1.append(ltz.lemmatize(word))
        final.append(ps.stem(word))
    # print(final2)
    if j==1: # give a string
        return " ".join(final)
    return final
text = "When I C++ . first-day \"met! 'her' will her's not she_was very quiet. She remained quiet during the entire two hour long journey from Stony Brook to New York."
x = TokStem(text,0)
print(x)

['c++', 'first-day', 'met', "'her", 'she_wa', 'quiet', 'remain', 'quiet', 'entir', 'two', 'hour', 'long', 'journey', 'stoni', 'brook', 'new', 'york']


In [ ]:
def TokStem(text,j): # Tokenise and Stem the given text
    words = [word for word in text.split() if word.lower() not in sw_nltk] # bag of tokenised words
    final = []
    for w in words:
        final.append(ps.stem(w))
    if j==1: # give a string
        return " ".join(final)
    return final
text = "When I first met her she was very quiet. She remained quiet during the entire two hour long journey from Stony Brook to New York."
x = TokStem(text,0)

In [5]:
def hbt(text): # convert HTML body to text
    x = BeautifulSoup(text).get_text()
    # print(x)
    return x

In [6]:
def rmTags(text): # convert tags to text
    st,out = [],[]
    for i in text:
        if i == '<':
            st = []
        elif i == '>':
            st += ' '
            out += st
        else:
            st += i
    return "".join(out[:-1]) # remove the last space

In [ ]:
############ test code
x = "<xz><y>"
x += "<w>"
print(x)
y = rmTag(x)
print(y,len(y))
print(x[:-1])

<xz><y><w>


NameError: ignored

## Loading the data 

In [7]:
df = pd.read_csv('Dataset.csv')
df

,RelatedPostId,OId,OTitle,OTags,OBody,DId,DTitle,DTags,DBody
0,4,4,How to convert a Decimal to a Double in C#?,<c#><floating-point><type-conversion><double><...,<p>I want to use a <code>Track-Bar</code> to c...,51027658,How to echo a JS variable to php?,<javascript><php>,<p>Is that possible to pass a JS variable to P...
1,9,9,How do I calculate someone's age based on a Da...,<c#><.net><datetime>,<p>Given a <code>DateTime</code> representing ...,2194999,How to calculate an age based on a birthday,<c#><asp.net-mvc><date-arithmetic>,<blockquote>\n<p><strong>Possible Duplicate:</...
2,11,11,Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-s...,<p>Given a specific <code>DateTime</code> valu...,7392566,"How to get datetime in words like ""today"", ""ye...",<php><zend-framework><datetime><datetime-format>,<blockquote>\n <p><strong>Possible Duplicate:...
3,13,13,Determine a user's timezone,<html><browser><timezone><user-agent><timezone...,<p>Is there a standard way for a web server to...,4746249,get user timezone,<php><javascript><timezone>,<blockquote>\n <p><strong>Possible Duplicate:...
4,14,14,Difference between Math.Floor() and Math.Trunc...,<.net><math>,"<p>What is the difference between <a href=""htt...",43174619,What is the difference in floor function and t...,<math>,<p>What is the difference between Math.Floor()...
...,...,...,...,...,...,...,...,...,...
7880,479799,479799,Replace host in Uri,<c#><.net><uri>,<p>What is the nicest way of replacing the hos...,49486804,using regex to find and replace hostname in url,<c#><regex>,<p>Currently I have my program able to achieve...
7881,479891,479891,What does the Call keyword do in VB6?,<vba><vb6><syntax>,<p>There's some code in our project that looks...,50789914,Calling Macro from inside Macro - Error Compil...,<vba><ms-office><ms-word><word-2013>,<p>I have been copying code moslty to work on ...
7882,479897,479897,How to remove duplicates from Python list and ...,<python><list><sorting><unique>,"<p>Given a list of strings, I want to sort it ...",32706281,Fastest way to remove duplicates from a list o...,<python><list><duplicates>,<p>How can I remove duplicate items from a lis...
7883,479923,479923,Is C# a single dispatch or multiple dispatch l...,<c#><programming-languages><multiple-dispatch>...,<p>I'm trying to understand what single and mu...,58463016,Why always this code will call the object para...,<c#><overloading>,<p>Why the below code would call the <code>pub...


## Data Preprocessing

In [8]:
N1 = len(df)
N2 = 300
# duplicate questions for training (with sorting)
Q2 = df[['OId','OTitle','OBody','OTags']].loc[0:N1-1].copy() # original qns
Q1 = df[['DId','DTitle','DBody','DTags']].loc[0:N2-1].copy() # doop qns
print("Duplicate  question set")
Q1


Duplicate  question set


,DId,DTitle,DBody,DTags
0,51027658,How to echo a JS variable to php?,<p>Is that possible to pass a JS variable to P...,<javascript><php>
1,2194999,How to calculate an age based on a birthday,<blockquote>\n<p><strong>Possible Duplicate:</...,<c#><asp.net-mvc><date-arithmetic>
2,7392566,"How to get datetime in words like ""today"", ""ye...",<blockquote>\n <p><strong>Possible Duplicate:...,<php><zend-framework><datetime><datetime-format>
3,4746249,get user timezone,<blockquote>\n <p><strong>Possible Duplicate:...,<php><javascript><timezone>
4,43174619,What is the difference in floor function and t...,<p>What is the difference between Math.Floor()...,<math>
...,...,...,...,...
295,3582464,Is there a XSD-driven random XML test data gen...,"<p>For stress tests, I would like to create XM...",<c#><java><xml><delphi><xsd>
296,14279410,Using List<T> vs. IList<T>,<blockquote>\n <p><strong>Possible Duplicate:...,<c#><asp.net><.net>
297,2195747,Code to create a password encrypted zip file?,<p>What is the Python code to create a passwor...,<python>
298,7932008,Can float values add to a sum of zero?,<blockquote>\n <p><strong>Possible Duplicate:...,<c++><c><floating-point>


In [ ]:
print("DataSet")
Q2

DataSet


,OId,OTitle,OBody,OTags
0,4,How to convert a Decimal to a Double in C#?,<p>I want to use a <code>Track-Bar</code> to c...,<c#><floating-point><type-conversion><double><...
1,9,How do I calculate someone's age based on a Da...,<p>Given a <code>DateTime</code> representing ...,<c#><.net><datetime>
2,11,Calculate relative time in C#,<p>Given a specific <code>DateTime</code> valu...,<c#><datetime><time><datediff><relative-time-s...
3,13,Determine a user's timezone,<p>Is there a standard way for a web server to...,<html><browser><timezone><user-agent><timezone...
4,14,Difference between Math.Floor() and Math.Trunc...,"<p>What is the difference between <a href=""htt...",<.net><math>
...,...,...,...,...
7880,479799,Replace host in Uri,<p>What is the nicest way of replacing the hos...,<c#><.net><uri>
7881,479891,What does the Call keyword do in VB6?,<p>There's some code in our project that looks...,<vba><vb6><syntax>
7882,479897,How to remove duplicates from Python list and ...,"<p>Given a list of strings, I want to sort it ...",<python><list><sorting><unique>
7883,479923,Is C# a single dispatch or multiple dispatch l...,<p>I'm trying to understand what single and mu...,<c#><programming-languages><multiple-dispatch>...


In [9]:

# # process the body wit tok, stem and modify tags and export

for i in range(Q2.shape[0]):
    
    x = TokStem(hbt(Q2.loc[i,'OBody']),1)
    y = rmTags(Q2.loc[i,'OTags'])
    z = TokStem(hbt(Q2.loc[i,'OTitle']),1)
    # print(x)
    # print(Q2.loc[i,'OBody'])
    # print("Y",y)
    # print(Q2.loc[i,'OTags'])
    # print(z)
    # print(Q2.loc[i,'OTitle'])


    Q2.loc[i,'OBody'] = x
    Q2.loc[i,'OTags'] = y
    Q2.loc[i,'OTitle'] =z
  
for i in range(Q1.shape[0]):
    x = TokStem(hbt(Q1.loc[i,'DBody']),1)
    y = rmTags(Q1.loc[i,'DTags'])
    z = TokStem(hbt(Q1.loc[i,'DTitle']),1)
    # print(x)
    # print(Q1.loc[i,'DBody'])
    # print("Y",y)
    # print(Q1.loc[i,'DTags'])
    # print(z)
    # print(Q1.loc[i,'DTitle'])


    Q1.loc[i,'DBody'] = x
    Q1.loc[i,'DTags'] = y
    Q1.loc[i,'DTitle'] =z

# Q2.to_csv('body_stem.csv') # export to CSV

# LDA processing

In [10]:
# also create master data for finding topics
data_words = []

for i in range(Q2.shape[0]):
  data_words.append((Q2.loc[i,'OBody']+Q2.loc[i,'OTitle']).split())

# create dictionary
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)
# LDA imp    
lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=100,minimum_probability=0.0)
lda_train = np.zeros((len(data_words),100)) # Q1 + Q2 size by 100 topics, refer to perplexity in paper

for i in range(len(data_words)): 
	lda_train[i] = np.array(lda_model[corpus[i]])[:,1]

# new = id2word.doc2bow(text)
# a = np.array(lda_model[text])[:,1]
# b = 

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


# Cosine Similarity between two strings

In [11]:
def cos_sim(a,b): # finding cosine similarity score for given 2 strings
    # input two strings that are split into words
    us = [] # creating union set
    us += b
    for x in a:
        if x not in b:
            us.append(x)
    freqa,freqb,frequ = dict(),dict(),dict()
    wa,wb = [], []
    for word in a: # freq bag for string a
        if word in freqa:
            freqa[word] += 1
        else:
            freqa[word] = 1
    for word in b:# freq bag for string b
        if word in freqb:
            freqb[word] += 1
        else:
            freqb[word] = 1
    for word in us: # freq bag for union of a and b
        if word in frequ:
            frequ[word] += 1
        else:
            frequ[word] = 1
    for i in range(len(us)): # calc TitleVec for a and b
        x = us[i]
        if x in a:
            wa.append(freqa[x]/frequ[x])
        else:
            wa.append(0)
        if x in b:
            wb.append(freqb[x]/frequ[x])
        else:
            wb.append(0)
    wa = np.array(wa)
    wb = np.array(wb)
    ans = np.dot(wa,wb)/(np.linalg.norm(wa)*np.linalg.norm(wb))
    if np.isnan(ans):
      return 0
    return  ans# cosine value
print(cos_sim(Q2.loc[0,'OTags'].split(),Q1.loc[0,'DTags'].split()))

0.0


## Calculating Similarity component scores

In [ ]:
print(SC[3000,30,1],SC[10,10,3],SC[50,50,3])

0.1513901837792264 0.014403460369729225 0.4769699368351657


In [ ]:
print(Q2.loc[0,'OTags'])
print(Q1.loc[0,'DTags'])


c# floating-point type-conversion double decimal
javascript php


In [ ]:
# b= ['OTitle','OBody','OTags']
# a = ['DTitle','DBody','DTags']
# SC = np.zeros([Q2.shape[0],Q1.shape[0],4]) # store all the component scores [ori,doop,4]
# y = 162130


# for i in range(Q2.shape[0]):
#     l11  = Q2.loc[i,b[0]].split()
#     l12  = Q2.loc[i,b[1]].split()
#     l13  = Q2.loc[i,b[2]].split()
#     for j in range(Q1.shape[0]):
#         l21  = Q2.loc[i,a[0]].split()
#         l22  = Q2.loc[i,a[1]].split()
#         l23  = Q2.loc[i,a[2]].split()

#         SC[i][j][0] = cos_sim(l11,l21) # TitleSC
#         SC[i][j][1] = cos_sim(l12,l22) # BodySC
#         SC[i][j][2] = cos_sim(Q2.loc[i,b[2]],Q1.loc[j,a[2]]) # TagsSC
b= ['OTitle','OBody','OTags']
a = ['DTitle','DBody','DTags']
SC = np.zeros([Q2.shape[0],Q1.shape[0],4]) # store all the component scores [ori,doop,4]
y = 162130
n2 = Q1.shape[0]
wa = np.zeros(100)
wb = np.zeros(100)
for i in range(Q2.shape[0]):
    print(i)
    for j in range(Q1.shape[0]):
        
        SC[i][j][0] = cos_sim(Q1.loc[j,a[0]].split(),Q2.loc[i,b[0]].split()) # TitleSC
        SC[i][j][1] = cos_sim(Q1.loc[j,a[1]].split(),Q2.loc[i,b[1]].split()) # BodySC
        SC[i][j][2] = cos_sim(Q1.loc[j,a[2]].split(),Q2.loc[i,b[2]].split()) # TagsSC
        text = (Q1.loc[j,'DBody']+Q1.loc[j,'DTitle']).split()
        corp = id2word.doc2bow(text)
        wa, wb = np.array(lda_model[corp])[:,1], lda_train[i] # lda train has Q1 qns and then Q2 qns
        SC[i][j][3] = np.dot(wa,wb)/(np.linalg.norm(wa)*np.linalg.norm(wb))# TopicSC
        if (np.isnan(SC[i][j][3])):
          SC[i][j][3] = 0
        if (j==35):
          print(SC[i][j])
np.save("Training set Similarity scores.npy",SC)


0
[0.         0.         0.         0.00133263]
1
[0.         0.10660036 0.         0.00382583]
2
[0.         0.07537784 0.         0.00183593]
3
[0.         0.07312724 0.         0.115021  ]
4
[0.         0.         0.         0.00317271]
5
[0.         0.         0.         0.00411387]
6
[0.         0.         0.         0.00195158]
7
[0.23570226 0.06154575 0.5        0.00229829]
8
[0.         0.05970814 0.         0.00120417]
9
[0.         0.05096472 0.         0.00142428]
10
[0.         0.         0.         0.00360581]
11
[0.         0.         0.         0.00142153]
12
[0.         0.05698029 0.         0.00151928]
13
[0.         0.14213381 0.         0.00190023]
14
[0.         0.         0.         0.00177946]
15
[0.         0.03183035 0.         0.00142398]
16
[0.         0.05170877 0.         0.00137848]
17
[0.33333333 0.08703883 0.70710678 0.0036311 ]
18
[0.         0.04828045 0.         0.00178845]
19
[0.         0.07106691 0.         0.00221102]
20
[0.         0.         0.  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


Streaming output truncated to the last 5000 lines.
5385
[0.         0.0855921  0.         0.89935698]
5386
[0.         0.         0.         0.00183775]
5387
[0.         0.1855674  0.         0.00192054]
5388
[0.         0.         0.         0.00215895]
5389
[0.         0.06428243 0.         0.00191187]
5390
[0.         0.02686077 0.         0.0013909 ]
5391
[0.         0.06428243 0.         0.00124468]
5392
[0.         0.06286946 0.         0.00169477]
5393
[0.         0.         0.         0.00281267]
5394
[0.         0.04445542 0.         0.002014  ]
5395
[0.         0.10987463 0.         0.00139146]
5396
[0.         0.09534626 0.70710678 0.00187274]
5397
[0.         0.03413944 0.         0.00147796]
5398
[0.         0.03015113 0.         0.00185279]
5399
[0.        0.        0.        0.0016021]
5400
[0.         0.         0.         0.00247434]
5401
[0.         0.         0.         0.00155421]
5402
[0.         0.13159034 0.         0.00163855]
5403
[0.         0.         0.     

# Estimating parameters

In [12]:
SC = np.load("Training set Similarity scores.npy")

In [ ]:
sp = np.linspace(0,1,2) # change last parameter for fine tuning
gco = 0 # global count

ga,gb,gc,gd = 0,0,0,0 # final parameter values
for a in sp:
    for b in sp:
        for c in sp:
            for d in sp:

                if a==0 and b==0 and c==0 and d==0:
                    continue
                tc = 0 # temp count
                for i in range(Q1.shape[0]): # iterate all duplicate qns
                    te = []
                    for j in range(Q2.shape[0]): # iterate over available qns
                        te.append(a*SC[j][i][0]+b*SC[j][i][1]+c*SC[j][i][2]+d*SC[j][i][3]) # composer score
                    x = te.index(max(te)) # get index of maximum element
                    if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
                        tc += 1
                if tc >= gco: # optimal value found
                    ga,gb,gc,gd = a,b,c,d
                    gco = tc
                    print(ga,gb,gc,gd,tc)
#             print(tc)
            
# print(SC[0][0:2])
# x = Q1.loc[0,a[1]]
# y = Q2.loc[0,b[0]]
# print(x,y)
# print(cos_sim(x,y))
# print(Q1.loc[0],Q1.loc[1])
# print(Q2.loc[0])

0.0 0.0 0.0 1.0 1
0.0 0.0 1.0 0.0 1


KeyboardInterrupt: ignored

In [ ]:
sp = np.linspace(0,1,2) # change last parameter for fine tuning
gco = 0 # global count
ga,gb,gc,gd = 0,0,0,0 # final parameter values
import random
k =  20

random_composers = np.zeros((100,5)) # store composer values, and recall rate
for iter in range(0,100):
  a = random.uniform(0,1)
  b = random.uniform(0,1)
  c = random.uniform(0,1)
  d = random.uniform(0,1)
  if a==0 and b==0 and c==0 and d ==0:
      continue
  tc = 0 # temp count
  count = 0
  de = {}
  for i in range(Q1.shape[0]): # iterate all duplicate qns
    te = []
    for j in range(Q2.shape[0]): # iterate over available qns
      te.append(a*SC[j][i][0]+b*SC[j][i][1]+c*SC[j][i][2] +d*SC[j][i][3]) # composer score
      de[j] = te[j]

    x = te.index(max(te)) # get index of maximum element
    # if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
    #   tc += 1

    temp_count= 0
    for key, val in sorted(de.items(), key= lambda item: item[1], reverse=True):
      
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count+=1
          break
        temp_count+=1
      else:
        break
    
  random_composers[iter] = [a,b,c,d,count]
  print(count)
  if count >= gco: # optimal value found
    ga,gb,gc,gd = a,b,c,d
    gco = count
    print(ga,gb,gc,gd,count)
#             print(tc)

np.save("composer_score_recall.npy",random_composers)

In [ ]:
arr = np.loadtxt("composer_score_recall_float.npy")
print(arr)
# np.savetxt("composer_score_recall_float.npy",arr, fmt='%f')

[[7.66942e-01 5.58142e-01 6.18187e-01 6.34700e-01 1.99000e+02]
 [2.17466e-01 1.05721e-01 1.32979e-01 2.32762e-01 1.62000e+02]
 [9.51983e-01 2.02783e-01 1.82110e-01 6.85130e-01 1.63000e+02]
 [7.38470e-01 3.30363e-01 6.36086e-01 3.34402e-01 2.11000e+02]
 [1.56423e-01 3.74802e-01 5.38216e-01 6.18922e-01 1.42000e+02]
 [6.82558e-01 6.49785e-01 1.05780e-01 6.58650e-02 2.20000e+02]
 [1.58002e-01 1.15691e-01 4.67325e-01 7.79302e-01 1.06000e+02]
 [5.53511e-01 3.03638e-01 9.15385e-01 4.70785e-01 1.92000e+02]
 [5.42153e-01 3.85910e-02 2.75130e-01 8.72434e-01 1.08000e+02]
 [7.87837e-01 5.04500e-02 6.64494e-01 6.73860e-02 2.01000e+02]
 [5.42881e-01 1.27414e-01 9.72143e-01 4.86363e-01 1.81000e+02]
 [7.41424e-01 7.03198e-01 1.13026e-01 8.03548e-01 1.37000e+02]
 [8.65406e-01 3.01960e-02 3.72710e-02 1.07926e-01 1.89000e+02]
 [8.48243e-01 2.40763e-01 2.63083e-01 5.70242e-01 1.82000e+02]
 [6.08890e-02 6.20750e-02 1.24436e-01 4.03780e-02 1.96000e+02]
 [7.41341e-01 1.18998e-01 6.38528e-01 4.07312e-01 2.050

In [19]:
ga = .8
gb = .51
gd = .37
gc = .01
print(ga,gb,gc,gd)

0.8 0.51 0.01 0.37


In [20]:
sp = np.linspace(0,1,2) # change last parameter for fine tuning
gco = 0 # global count
# ga,gb,gc = 0,0,0 # final parameter values

# a = 0.81
# b = 0.91
# c = 0.57
a = .89
b = .966
c = .50
d = 0.03
# if a==0 and b==0 and c==0:
    # continue
tc = 0 # temp count
k =  5
count = 0
de = {}
for i in range(Q1.shape[0]): # iterate all duplicate qns
    te = []
    for j in range(Q2.shape[0]): # iterate over available qns
        te.append(ga*SC[j][i][0]+gb*SC[j][i][1]+gc*SC[j][i][2] + gd*SC[j][i][3]) # composer score
        de[j] = te[j]

    x = te.index(max(te)) # get index of maximum element
    if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
        tc += 1
    temp_count= 0
    for key, val in sorted(de.items(), key= lambda item: item[1], reverse=True):
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count+=1
          break
        temp_count+=1
      else:
        break

In [21]:
print(count)

144


## Validating the composer model

In [ ]:
tc = 0 # temp count
k =  20
count = 0
de = {}
print(ga,gb,gc,gd)
original_data = df[['OId','OTitle','OBody','OTags']].loc[300:1500-1].copy() # original qns
test_data = df[['DId','DTitle','DBody','DTags']].loc[300:1500-1].copy() # doop qns
for i in range(300,300+test_data.shape[0]): # iterate all duplicate qns
    te = []
    # print(test_data.loc[i,'DTags'])
    x = TokStem(hbt(test_data.loc[i,'DBody']),0)
    y = rmTags(test_data.loc[i,'DTags']).split()
    z = TokStem(hbt(test_data.loc[i,'DTitle']),0)
    
    text = (x+ y)

    corp = id2word.doc2bow(text)

    wa, wb = np.array(lda_model[corp])[:,1], lda_train[i]
        
    b= ['OTitle','OBody','OTags']
    for j in range(Q2.shape[0]): # iterate over available qns
        wa, wb = np.array(lda_model[corp])[:,1], lda_train[j] # lda train has Q1 qns and then Q2 qns
        topic_similarity = np.dot(wa,wb)/(np.linalg.norm(wa)*np.linalg.norm(wb))# TopicSC
        if (np.isnan(topic_similarity)):
          topi_similarity = 0
        te.append(ga*cos_sim(z,Q2.loc[j,b[0]].split()) + gb*cos_sim(x,Q2.loc[j,b[1]].split())+ gc*cos_sim(y,Q2.loc[j,b[2]].split()) + gd*topic_similarity) # composer score
        de[j] = te[j]

    # x = te.index(max(te)) # get index of maximum element
    # if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
    #     tc += 1
    temp_count= 0
    for key, val in sorted(de.items(), key= lambda item: item[1], reverse=True):
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count+=1
          break
        temp_count+=1
      else:
        break
    print(count)
print("Accuracy = ", count/1200)


0.632718670527089 0.9676276088699085 0.43945065835845154 0.1762235833261695


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


1
1
2
3
4
5
6
7
7
7
8
9
10


In [ ]:
print(172/200)

0.86


## Return top k questions

In [ ]:
tc = 0 # temp count
k =  20
count = 0
de = {}
print(ga,gb,gc)
no = np.random.randint(300,500)
original_data = df[['OId','OTitle','OBody','OTags']].loc[no].copy() # original qns
test_data = df[['DId','DTitle','DBody','DTags']].loc[no].copy() # doop qns

te = []
# print(test_data.loc[i,'DTags'])
x = TokStem(hbt(df.loc[no,'DBody']),0)
y = rmTags(df.loc[no,'DTags']).split()
z = TokStem(hbt(df.loc[no,'DTitle']),0)
# print(x)
# print(y)
# print(z)
# break
b= ['OTitle','OBody','OTags']
# print(b)
# print(ga,gb,gc)
for j in range(Q2.shape[0]): # iterate over available qns
    te.append(ga*cos_sim(z,Q2.loc[j,b[0]].split()) + gb*cos_sim(x,Q2.loc[j,b[1]].split())+ gc*cos_sim(y,Q2.loc[j,b[2]].split())) # composer score
    de[j] = te[j]
# x = te.index(max(te)) # get index of maximum element
# if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
#     tc += 1
temp_count= 0
print("------------ Given question ------")
print(no)
print("Title -->  ",df.loc[no,'DTitle'])
print("Question --> ", df.loc[no,'DBody'])
print("Tags --> ", df.loc[no,'DTags'])
print("-----")
for key, val in sorted(de.items(), key= lambda item: item[1], reverse=True):
  print(" ----- Question ---- ", temp_count)
  print("Title --->", df.loc[key,'OTitle'])
  print("Question --->", df.loc[key,'OBody'])
  print("Tags --->", df.loc[key,'OTags'])
  print(Q2.loc[key], df.loc[no])
  print("----")
  if (temp_count<k):
    if Q2.loc[key,'OId'] == df.loc[no,'OId']:
      print(" ----- Question ---- ", temp_count)
      print("Title --->", df.loc[key,'OTitle'])
      print("Question --->", df.loc[key,'OBody'])
      print("Tags --->", df.loc[key,'OTags'])
      print("----")
          
      count+=1
      break
    temp_count+=1
  else:
    break
print(count)
print(tc,count)
if tc >= gco: # optimal value found
    ga,gb,gc = a,b,c
    gco = tc
    print(ga,gb,gc,tc)

0.632718670527089 0.9676276088699085 0.43945065835845154
['accord', 'geeksforgeeks.org/encapsulation-in-java', 'encapsul', '=', 'data', 'hide', '+', 'abstract', 'ye', 'see', 'data', 'hide', 'abstract', 'peopl', 'give', 'exampl', 'encapsul', 'class', 'person', 'privat', 'string', 'name', 'privat', 'int', 'age', 'public', 'void', 'setnam', 'string', 'name', 'this.nam', '=', 'name', 'public', 'string', 'getnam', 'return', 'this.nam', 'public', 'void', 'setag', 'int', 'age', 'this.ag', '=', 'age', 'public', 'int', 'getag', 'return', 'this.ag', 'say', 'exampl', 'encapsul', 'ref', 'http', '//www.youtube.com/watch', 'v=tjyzwqjknpc', 'also', 'person', 'teach', 'video', 'realli', 'good', 'java', 'difficult', 'believ', 'wrong', 'realli', 'encapsul', 'data', 'hide', '+', 'abstract', 'group', 'variabl', 'method', 'singl', 'unit', 'definit', 'correct', 'answer', 'know', 'otherwis', 'ignor', 'question', 'tag', 'whatev', 'new', 'stackoverflow', 'dont', 'know', 'rule', 'ask', 'question', 'want', 'clea

SyntaxError: ignored

In [ ]:
print(Q2.loc[key])
print(key)
print(df.loc[324])

OId                                                  361336
OTitle                                    gener collect c#?
OBody     i'm tri build first gener list run problems. u...
OTags                                      c# .net generics
Name: 317, dtype: object
317
OId                                                        361336
CreationDate                                  2008-12-11 23:02:57
OTitle                        What are Generic Collections in C#?
OBody           <p>I'm trying to build my first generic list a...
OTags                                        <c#><.net><generics>
DId                                                       9857180
DTitle                                 What does <T> denote in C#
DBody           <p>I'm new to C# and directly diving into modi...
DTags                                        <c#><.net><generics>
Name: 324, dtype: object


In [ ]:
print(df.loc[no])

OId                                                        361336
CreationDate                                  2008-12-11 23:02:57
OTitle                        What are Generic Collections in C#?
OBody           <p>I'm trying to build my first generic list a...
OTags                                        <c#><.net><generics>
DId                                                       9857180
DTitle                                 What does <T> denote in C#
DBody           <p>I'm new to C# and directly diving into modi...
DTags                                        <c#><.net><generics>
Name: 324, dtype: object
